In [49]:
import os
import warnings
import findfile
import pandas as pd

In [50]:
import transformers
print(transformers.__version__)

4.28.0


In [51]:
import torch

In [52]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060 Ti'

In [6]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [86]:
from pyabsa import ABSAInstruction as absa_instruction

In [87]:
warnings.filterwarnings("ignore")

In [88]:
task_name = "multitask"
experiment_name = "instruction"
model_checkpoint = "google/flan-t5-base"

In [89]:
print("Experiment Name: ", experiment_name)
model_out_path = "checkpoints"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

Experiment Name:  instruction
Model output path:  checkpoints\multitask\googleflan-t5-base-instruction


In [115]:
id_train_file_path = "./ABSADatasets/datasets/acos_datasets/300.Hotel"
id_test_file_path = "./ABSADatasets/datasets/acos_datasets/300.Hotel"

In [116]:
id_tr_df = absa_instruction.data_utils.read_json(id_test_file_path, "train")

./ABSADatasets/datasets/acos_datasets/300.Hotel\train.jsonl


In [100]:
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

./ABSADatasets/datasets/acos_datasets/301.Hotel\test.jsonl


In [101]:
id_tr_df = pd.DataFrame(id_tr_df)


In [102]:
id_te_df = pd.DataFrame(id_te_df)

In [103]:
print(id_tr_df)

                                                   text  \
0                 But the staff was so horrible to us .   
1     To be completely fair , the only redeeming fac...   
2     The food is uniformly exceptional , with a ver...   
3     Our agreed favorite is the orrechiete with sau...   
4     The Bagels have an outstanding taste with a te...   
...                                                 ...   
1261  The selection changes frequently but the basic...   
1262  I have to say that if this what makes it easie...   
1263  We were told that they were booked solid and n...   
1264  After dealing with subpar pizza all over the K...   
1265  Each table has a pot of boiling water sunken i...   

                                                 labels  
0     [{'aspect': 'staff', 'opinion': 'horrible', 'p...  
1     [{'aspect': 'food', 'opinion': 'above average'...  
2     [{'aspect': 'food', 'opinion': 'exceptional', ...  
3     [{'aspect': 'orrechiete with sausage and chick...  
4

In [95]:
print(id_te_df)

Empty DataFrame
Columns: []
Index: []


In [15]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [16]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [17]:
# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

[2024-04-09 15:47:59] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-04-09 15:47:59] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-04-09 15:47:59] (2.4.1.post1) Checkpoint:google/flan-t5-base is not found, you can raise an issue for requesting shares of checkpoints
[2024-04-09 15:47:59] (2.4.1.post1) No checkpoint found in Model Hub for task: google/flan-t5-base


In [18]:
# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 2135/2135 [00:00<00:00, 13558.80 examples/s]


In [21]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map:   0%|          | 0/5601 [00:00<?, ? examples/s]

Map:   0%|          | 0/2135 [00:00<?, ? examples/s]

In [19]:
# Training arguments
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 6,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 6,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    # 'fp16': True,
    "fp16": False,
}


In [20]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


  0%|          | 0/5604 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
 17%|█▋        | 934/5604 [04:04<18:26,  4.22it/s]

{'eval_loss': 0.2084820568561554, 'eval_runtime': 22.7298, 'eval_samples_per_second': 93.93, 'eval_steps_per_second': 5.895, 'epoch': 1.0}


                                                     
 33%|███▎      | 1868/5604 [07:56<12:00,  5.19it/s]

{'eval_loss': 0.16863885521888733, 'eval_runtime': 19.6716, 'eval_samples_per_second': 108.532, 'eval_steps_per_second': 6.812, 'epoch': 2.0}


...

{'eval_loss': 0.1823197454214096, 'eval_runtime': 22.6865, 'eval_samples_per_second': 94.109, 'eval_steps_per_second': 5.907, 'epoch': 5.0}


                                                     
100%|██████████| 5604/5604 [23:40<00:00,  4.59it/s]

{'eval_loss': 0.18908771872520447, 'eval_runtime': 23.039, 'eval_samples_per_second': 92.669, 'eval_steps_per_second': 5.816, 'epoch': 6.0}


100%|██████████| 5604/5604 [23:42<00:00,  3.94it/s]


{'train_runtime': 1422.4996, 'train_samples_per_second': 23.625, 'train_steps_per_second': 3.94, 'train_loss': 0.2370254331448519, 'epoch': 6.0}


In [21]:
# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="train",
    batch_size=16,
)
id_tr_labels = [i.strip() for i in id_ds["train"]["labels"]]

Prediction from trainer


100%|██████████| 351/351 [03:34<00:00,  1.63it/s]


In [23]:
id_te_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="test",
    batch_size=16,
)
id_te_labels = [i.strip() for i in id_ds["test"]["labels"]]

Prediction from trainer


100%|██████████| 134/134 [01:17<00:00,  1.74it/s]


In [22]:
metrics = t5_exp.get_classic_metrics(id_tr_labels, id_tr_pred_labels)
print("----------------------- Classic Training Set Metrics -----------------------")
print(metrics)

NULL
NULL
NULL:negative
NULL:negative
NULL:overpriced
NULL:overpriced
NULL
NULL
NULL:positive
NULL:positive
NULL:conveniently
NULL:conveniently
drinkmenu
drinkmenu
drinkmenu:positive
drinkmenu:positive
drinkmenu:love
drinkmenu:love
drinkmenu:DRINKS#STYLE_OPTIONS
drinkmenu:DRINKS#STYLE_OPTIONS
food|decor
food|decor
food:positive|decor:neutral
food:positive|decor:positive
food:great|food:great|decor:NULL
food:great|decor:NULL
food:FOOD#QUALITY|food:FOOD#PRICES|decor:AMBIENCE#GENERAL
food:FOOD#QUALITY|decor:AMBIENCE#GENERAL
dessert|specialroll|regularroll
specialroll|regularroll
dessert:positive|specialroll:positive|regularroll:positive
dessert:positive|specialroll:positive|regularroll:positive
dessert:saveroom|specialroll:enough|regularroll:enough
dessert:NULL|specialroll:NULL|regularroll:NULL
dessert:FOOD#QUALITY|specialroll:FOOD#STYLE_OPTIONS|regularroll:FOOD#STYLE_OPTIONS
dessert:FOOD#QUALITY|specialroll:FOOD#QUALITY|regularroll:FOOD#QUALITY
NULL
NULL
NULL:negative
NULL:negative
NULL:

In [25]:
metrics = t5_exp.get_classic_metrics(id_te_labels, id_te_pred_labels)
print("----------------------- Classic Testing Set Metrics -----------------------")
print(metrics)

NULL
NULL
NULL:positive
NULL:positive
NULL:yum
NULL:yum
sushi
sushi
sushi:positive
sushi:positive
sushi:good
sushi:good
sushi:FOOD#QUALITY
sushi:FOOD#QUALITY
portions
portions
portions:neutral
portions:positive
portions:notthebiggest|portions:adequate
portions:notthebiggest|portions:adequate
portions:FOOD#STYLE_OPTIONS|portions:FOOD#STYLE_OPTIONS
portions:FOOD#QUALITY
greenteacremebrulee
greenteacremebrulee
greenteacremebrulee:positive
greenteacremebrulee:positive
greenteacremebrulee:must
greenteacremebrulee:must
greenteacremebrulee:FOOD#QUALITY
greenteacremebrulee:FOOD#QUALITY
sushi|service
sushi|service
sushi:positive|service:positive
sushi:positive|service:positive
sushi:great|service:better
sushi:great|service:better
sushi:FOOD#QUALITY|service:SERVICE#GENERAL
sushi:FOOD#QUALITY|service:SERVICE#GENERAL
staff
staff
staff:positive
staff:positive
staff:accomodating
staff:accomodating
staff:SERVICE#GENERAL
staff:SERVICE#GENERAL
restaurant
restaurant
restaurant:positive
restaurant:positi